# 1- **Content** based Restaurant Recommendation based on reviews

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!unzip /content/drive/MyDrive/zomato.csv.zip

Archive:  /content/drive/MyDrive/zomato.csv.zip
  inflating: zomato.csv              


In [108]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [109]:
df = pd.read_csv('zomato.csv')

In [110]:
data = df[['name', 'reviews_list']]

In [111]:
data.drop_duplicates(inplace = True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [112]:
data = data[data['reviews_list'] != "[]"]

In [114]:
#combining 
data = data.groupby('name').agg({'reviews_list': ' '.join})

In [13]:
import requests
 
print('Download Starting...')
 
url = 'https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz'
 
r = requests.get(url)
 
filename = url.split('/')[-1] # this will take only -1 splitted part of the url
 
with open(filename,'wb') as output_file:
    output_file.write(r.content)

Download Starting...


In [14]:
from tqdm import tqdm
from gensim import models
model = models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)

def word_to_vec(data,fea):
    #removing symbols
    data[fea]=data[fea].str.replace(r"(\d+|\W+)", ' ',regex=True)
    print("SYMBOLS REMOVED")
    #removing 2 letters words
    data[fea]=data[fea].str.replace(r"\b\w{1,2}\b", '',regex=True)
    print("2 letter words REMOVED")
    #removing extra spaces
    for i in range(4):
        data[fea]=data[fea].str.replace(" "*2, ' ',regex=True)
    print("EXTRA SPACES REMOVED")
    #word to vec
    print("STARTING Word to vec")
    #model = models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)
    # print(" TITLE text to W2V conversion Started...")
    L=[]
    for i in tqdm(data[fea]):
        sent=i.split(" ")
        w2v=[]
        for word in sent:
            try:
                w2v.append(model[word])
            except: pass
        try: array2= sum(w2v)/len(w2v)
        except: array2= np.zeros(300)
        L.append(np.array(array2))
        del sent
    #dataframe
    df=pd.DataFrame(np.array(L))
    return df

In [115]:
dataW = word_to_vec(data,'reviews_list')

SYMBOLS REMOVED
2 letter words REMOVED
EXTRA SPACES REMOVED
STARTING Word to vec


100%|██████████| 7041/7041 [02:22<00:00, 49.28it/s]


In [116]:
dataW.shape

(7041, 300)

In [117]:
name = list(data.index)
vector = list(np.array(dataW))
dic = dict(zip(name, vector))

In [118]:
from numpy import dot
from numpy.linalg import norm

def top_recom(restaurantName, i):
  s = []
  v1 = dic[restaurantName]
  n = list(dic.keys())
  v = list(dic.values())
  for v2 in v:
    sim = dot(v1, v2)/(norm(v1)*norm(v2))
    s.append(sim)
  sorted_sim = np.argsort(np.array(s))[::-1][:i]
  return np.array(n)[list(sorted_sim)], s

In [132]:
a, b = top_recom("Jalsa", 5)

In [133]:
a

array(['Jalsa', 'Jalsa Gold', 'Silbatti', 'Ji Hazoor', 'Punjabi Times'],
      dtype='<U159')

# 2- Collaborative filtering Recommendation System

In [31]:
! unzip /content/drive/MyDrive/zomato.csv.zip

Archive:  /content/drive/MyDrive/zomato.csv.zip
  inflating: zomato.csv              


In [32]:
import pandas as pd

zf = pd.read_csv('zomato.csv')

In [33]:
import re
pattern = '(.*)'
for i in zf['reviews_list']:
  print(*(re.findall(pattern, i)), sep='\n')
  break

[('Rated 4.0', 'RATED\n  A beautiful place to dine in.The interiors take you back to the Mughal era. The lightings are just perfect.We went there on the occasion of Christmas and so they had only limited items available. But the taste and service was not compromised at all.The only complaint is that the breads could have been better.Would surely like to come here again.'), ('Rated 4.0', 'RATED\n  I was here for dinner with my family on a weekday. The restaurant was completely empty. Ambience is good with some good old hindi music. Seating arrangement are good too. We ordered masala papad, panner and baby corn starters, lemon and corrionder soup, butter roti, olive and chilli paratha. Food was fresh and good, service is good too. Good for family hangout.\nCheers'), ('Rated 2.0', 'RATED\n  Its a restaurant near to Banashankari BDA. Me along with few of my office friends visited to have buffet but unfortunately they only provide veg buffet. On inquiring they said this place is mostly visi

In [34]:
# To seperate rate and reviews from the reviews

import re
from tqdm import tqdm
rate = []
reviews = []
restName = zf['name']
loc = zf['reviews_list']
name = []
f=0

for idx, row in tqdm(enumerate(zip(loc, restName))):
  i, r = row
  temp = list(re.findall("\((.*?)\)", i))
  for j in temp:
    if "Rated" not in j or "RATED" not in j:
      continue
    try:
      try:
        a, b = (j).split("RATED\\n")
      except:
        a, b = (j).split("RATED\n")
      
      rate.append(float(a.split("Rated")[1][:3]))
      
      reviews.append(b[:len(b)-1])
      name.append(r)
    except Exception as e:
      print(e, idx, j)
      print((j.split("', 'RATED\\n ")))
      f=1
      break
  if f==1:
    break

51717it [00:13, 3963.92it/s]


In [35]:
data = pd.DataFrame()

data['reviews'] = reviews
data['name'] = name
data['rate'] = rate

In [37]:
data.to_csv('/content/drive/MyDrive/pureZomato.csv')

In [36]:
# Cleaning the text
import nltk
from nltk.corpus import stopwords 
nltk.download('stopwords')
import string
import re
def clean_text(text):
    text = text.translate(string.punctuation)
    
    ## Convert words to lower case and split them
    text = text.lower().split()
    
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]
    
    text = " ".join(text)
    
    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text) 
    text = re.sub(r"!", " ", text)
    text = re.sub("(\s\d+)","",text)
    text = re.sub(r":", " ", text)
    text = re.sub(r"\s\S*\d{1,}\S*\s", " ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ", text)
    text = re.sub(r"\+", " ", text)
    text = re.sub(r"\-", " ", text)
    text = re.sub(r"\=", " ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    #removing symbols
    text = re.sub(r"(\d+|\W+)", ' ', text)
    #data[fea]=data[fea].str.replace(r"(\d+|\W+)", ' ',regex=True)
    print("SYMBOLS REMOVED")
    #removing 2 letters words
    text = re.sub(r"\b\w{1,2}\b", '', text)
    #data[fea]=data[fea].str.replace(r"\b\w{1,2}\b", '',regex=True)
    print("2 letter words REMOVED")
    #removing extra spaces
    for i in range(4):
      text = re.sub(" "*2, ' ', text)
    print("EXTRA SPACES REMOVED")    
    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [37]:
#text = []
#for i in (data.reviews):
#  text.append(clean_text(i))

data['reviews'] = data['reviews'].apply((clean_text))

Streaming output truncated to the last 5000 lines.
2 letter words REMOVED
EXTRA SPACES REMOVED
SYMBOLS REMOVED
2 letter words REMOVED
EXTRA SPACES REMOVED
SYMBOLS REMOVED
2 letter words REMOVED
EXTRA SPACES REMOVED
SYMBOLS REMOVED
2 letter words REMOVED
EXTRA SPACES REMOVED
SYMBOLS REMOVED
2 letter words REMOVED
EXTRA SPACES REMOVED
SYMBOLS REMOVED
2 letter words REMOVED
EXTRA SPACES REMOVED
SYMBOLS REMOVED
2 letter words REMOVED
EXTRA SPACES REMOVED
SYMBOLS REMOVED
2 letter words REMOVED
EXTRA SPACES REMOVED
SYMBOLS REMOVED
2 letter words REMOVED
EXTRA SPACES REMOVED
SYMBOLS REMOVED
2 letter words REMOVED
EXTRA SPACES REMOVED
SYMBOLS REMOVED
2 letter words REMOVED
EXTRA SPACES REMOVED
SYMBOLS REMOVED
2 letter words REMOVED
EXTRA SPACES REMOVED
SYMBOLS REMOVED
2 letter words REMOVED
EXTRA SPACES REMOVED
SYMBOLS REMOVED
2 letter words REMOVED
EXTRA SPACES REMOVED
SYMBOLS REMOVED
2 letter words REMOVED
EXTRA SPACES REMOVED
SYMBOLS REMOVED
2 letter words REMOVED
EXTRA SPACES REMOVED
SYMBO

In [38]:
data.to_csv('/content/drive/MyDrive/kulZomato.csv')

In [41]:
import pandas as pd

abhi = pd.read_csv('/content/drive/MyDrive/kulZomato.csv', index_col=0)

/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [42]:
abhi.head(10)

,reviews,name,rate
0,beautiful place dine the interiors take back m...,Jalsa,4.0
1,dinner family weekday restaurant completely em...,Jalsa,4.0
2,restaurant near banashankari bda along office ...,Jalsa,2.0
3,went weekend one buffet two took ala carte fir...,Jalsa,4.0
4,best thing place ambiance second best thing yu...,Jalsa,5.0
5,great food pleasant ambience expensive coll pl...,Jalsa,5.0
6,good ambience tasty food ncheese chilli parath...,Jalsa,4.0
7,can wrong jalsa never fan buffet thus always o...,Jalsa,4.0
8,overdelighted service food provided place roya...,Jalsa,5.0
9,place nice comfortable food wise jalea outlets...,Jalsa,4.0


In [43]:
abhi['reviews'].duplicated().sum()

1192340

In [45]:
# it will remove all the duplicates
abhi.drop_duplicates(inplace=True,)

In [46]:
# this will show 5399 duplicate reviews, but they are not duplicate.
# they are small reviews like good, very good. So no need to remove
abhi['reviews'].duplicated().sum()

6946

In [52]:
abhi.shape

(132299, 3)

In [56]:
len(abhi['name'].unique()), len(abhi['reviews'].unique())

(7041, 125353)

In [58]:
abhi['rate'].value_counts()

4.0    41078
5.0    37412
3.0    24347
1.0    20078
2.0     9384
Name: rate, dtype: int64

In [61]:
abhi.dropna(inplace = True)

In [62]:
from tqdm import tqdm
uName = []
noRev = []
for i in tqdm(abhi['reviews']):
  try:
    noRev.append(len(i.split(" ")))
  except:
    print(i)

100%|██████████| 132004/132004 [00:00<00:00, 264087.96it/s]


In [71]:
len(list(filter(lambda x: x <= 22, noRev)))

66121

In [72]:
new = list(map(lambda x: x <= 22, noRev))

In [74]:
abhi = abhi[new]

In [76]:
len(abhi['reviews'].unique())

59722

In [64]:
#sum(noRev)/len(noRev)
import numpy as np
np.median(np.array(noRev))
#np.min(np.array(noRev))

22.0

In [42]:
data.to_csv("/content/drive/MyDrive/CleanZomatoData.csv")

In [28]:
import re
def word_to_vec(text):
    #removing symbols
    text = re.sub(r"(\d+|\W+)", ' ', text)
    #data[fea]=data[fea].str.replace(r"(\d+|\W+)", ' ',regex=True)
    print("SYMBOLS REMOVED")
    #removing 2 letters words
    text = re.sub(r"\b\w{1,2}\b", '', text)
    #data[fea]=data[fea].str.replace(r"\b\w{1,2}\b", '',regex=True)
    print("2 letter words REMOVED")
    #removing extra spaces
    for i in range(4):
      text = re.sub(" "*2, ' ', text)
    print("EXTRA SPACES REMOVED")
    return text

In [24]:
abhi['reviews'][0]

'  A beautiful place to dine in.The interiors take you back to the Mughal era. The lightings are just perfect.We went there on the occasion of Christmas and so they had only limited items available. But the taste and service was not compromised at all.The only complaint is that the breads could have been better.Would surely like to come here again.'

In [29]:
word_to_vec(abhi['reviews'][0])

SYMBOLS REMOVED
2 letter words REMOVED
EXTRA SPACES REMOVED


' beautiful place dine The interiors take you back the Mughal era The lightings are just perfect went there the occasion Christmas and they had only limited items available But the taste and service was not compromised all The only complaint that the breads could have been better Would surely like come here again '

In [77]:
rating_matrix = pd.pivot_table(abhi, values='rate', index=['name'], columns=['reviews'])
rating_matrix.shape

(5840, 59722)

In [79]:
rating_matrix.head()

reviews,,adyar amanda bhavan restaurant really great looking like please also nice nfood quality good testing service excellent great experience thanks,agent jacks mumbai many times one bangalore good too pasta soooo good place pasta,aishwarya parkland gone service nowadays understandable much orders otherwise late rates okay extremes nsit talk green signal,all services all food absolutely useless staff came hungry still sitting waiting food cos dumbass took order managed completely forget order,ally ally like place,almost everything here good hygienic there seating arrangement here meat usually well done melts mouth,almost years since going place never fails disappoint lassi fruit sandae must try seasonal mango lassi strawberry ice cream,already surrender telephone new connection till date given connection iyer,always game trying food specially biryani,always looking place enjoy old visited place quick metro train ride son big fan train,always loved shangri entering elevator reachingth floor staff asks one restaurant would like yataii saffron capres,always stands kind service provide variety dishes serve customize every dish least starter,always wide varieties appetizing sweets savories always courteous serves tasty food items consumers great job keep going like this ngoo,amanah expanded quality went while thats found place shawarmas good rolls alamanah back dont visit often,amazeballs one best rolls eaten bangalore keep rocking guys would love eat wish run outlet forever may eat rolls forever,amazing food best online order night may allah swt make successful aameen aameen aameen maashaallah subahaanallah walhamdulillah wallaahu akbar,ambiance service food dishes less quantit,ambience food service prompt coming beers loved brews belgian wit belgian tripple highly recommend place,ambience perfect dhaba feeling food average good music friendly staff nportion way less make big hole pocket good visit that,ambiencegreat locationgreat serviceok foodok avoid tandoori mushroo,amitgera really love food needs change seriously disliked regular good rapport neil really worth umbrella,amma pastries located jayanagarth block cool place variety flavoured pastries nwhat,around familie,authentic expected garlic chicken good delivery time accurate said trying place first time ended good expected keep guys,average place average italian cuisine one visit fine can say food great manageable ambiance clumsy service fine,average rating resturant made little skeptical ordering here food average say good great surely average rating,away home search ghar khana ended here thanks dillibite,awesome,awesome fine restaurant would like recommend every one need hunger first thanks quickest delivery malesh,awesome french fries paratha good taste good packing,bakery decent prices nhave ordered bake quite times never disappointed love fresh fruit cake also dark chocolate one try,beautiful thing comes small pack place reminds primitive india beauty kashmir place got puriety within makes taste relaxation food,been regular ice cream joint college days place good seating place fast service wishing change menu style presentation,beer food vibe music loved every bit experience xooxbrewmill nthe place going top must visit places heard first,beir cafe chilled place vega mall serving foods freindly staff outside sittings brave malt rock beverage try nice area,best litchi flavored cake moms birthdays sooo tasty remembered years loved pasta also nice,best malabar cuisines snacks nand thalassery dum biriyani ncool place hangout ncheap best koramangala,best nice calm place good music chill peace friends disappointed limited stock kingfisher beer cocktail chilled either thin texture,best sweet savory store bangalore one must definitely try milk ghee sweets simply superb time favorites ngoo,...,zomato always good service helpful besides occasionally getting lost occasionall,zomato customer try biriyani think come its rupees biriyani quantity taste also good thanks zomato thank

In [ ]:
D1 = abhi[['name', 'reviews']]

In [81]:
newDf = abhi.groupby('name').agg({'reviews': ' '.join})

In [82]:
newDf.shape

(5840, 1)

In [102]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import WordPunctTokenizer

#userid vectorizer
review_vectorizer = TfidfVectorizer(tokenizer = WordPunctTokenizer().tokenize, max_features=1000)
review_vectors = review_vectorizer.fit_transform(abhi['reviews'].unique())
review_vectors.shape

(59722, 1000)

In [103]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import WordPunctTokenizer

#userid vectorizer
name_vectorizer = TfidfVectorizer(tokenizer = WordPunctTokenizer().tokenize, max_features=1000)
name_vectors = name_vectorizer.fit_transform(pd.DataFrame(newDf)['reviews'])
name_vectors.shape

(5840, 1000)

In [105]:
P = pd.DataFrame(name_vectors.toarray(), index=newDf.index, columns=name_vectorizer.get_feature_names())
Q = pd.DataFrame(review_vectors.toarray(), index=abhi.reviews.unique(), columns=review_vectorizer.get_feature_names())

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [106]:
P.head(2)

,able,absolutely,according,actually,add,added,affordable,afternoon,again,all,almost,along,aloo,already,also,although,always,amazing,ambiance,ambience,amount,and,andhra,another,anyone,anything,apart,app,appreciate,are,area,around,ask,asked,ate,atleast,atmosphere,authentic,available,average,...,weekends,well,went,whenever,which,white,whitefield,whole,wide,will,wings,wise,wish,with,within,without,wonderful,word,work,working,world,worse,worst,worth,worthy,would,wow,wrap,wrong,xad,year,years,yes,yesterday,yet,you,yum,yumm,yummy,zomato
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
#FeelTheROLL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.044665,0.0,0.0,0.101711,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.142856,0.0,...,0.0,0.1552,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.10821,0.0,0.164252,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.063752,0.0
#L-81 Cafe,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.245929,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


In [107]:
def matrix_factorization(R, P, Q, steps=25, gamma=0.001,lamda=0.02):
    for step in range(steps):
        for i in R.index:
            for j in R.columns:
                if R.loc[i,j]>0:
                    eij=R.loc[i,j]-np.dot(P.loc[i],Q.loc[j])
                    P.loc[i]=P.loc[i]+gamma*(eij*Q.loc[j]-lamda*P.loc[i])
                    Q.loc[j]=Q.loc[j]+gamma*(eij*P.loc[i]-lamda*Q.loc[j])
        e=0
        for i in R.index:
            for j in R.columns:
                if R.loc[i,j]>0:
                    e= e + pow(R.loc[i,j]-np.dot(P.loc[i],Q.loc[j]),2)+lamda*(pow(np.linalg.norm(P.loc[i]),2)+pow(np.linalg.norm(Q.loc[j]),2))
        if e<0.001:
            break
        
    return P,Q

In [108]:
%%time
p, q = matrix_factorization(rating_matrix, P, Q, steps=25, gamma=0.001,lamda=0.02)

KeyboardInterrupt: ignored

In [109]:
# Store P, Q and vectorizer in pickle file
import pickle
output = open('/content/drive/MyDrive/yelp_recommendation_model_8.pkl', 'wb')
pickle.dump(p,output)
pickle.dump(q,output)
pickle.dump(name_vectorizer,output)
output.close()

NameError: ignored